In [86]:
from transformers import pipeline
from nltk import sent_tokenize # tokenize sequence: Separate text -> multiple sentences
import nltk
import torch
from glob import glob #get all files path we want
import pandas as pd
import numpy as np

In [87]:
#This model is used by NLTK for tasks like splitting text into sentences.
nltk.download('punkt_tab')#pre-trained tokenizer model 

[nltk_data] Downloading package punkt_tab to /home/kelvin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# I. Loading Model

In [88]:
model_name = 'facebook/bart-large-mnli'
device = 0 if torch.cuda.is_available() else 'cpu'

In [89]:
def load_model(device):
    theme_classifier = pipeline(
        'zero-shot-classification',
        model = model_name,
        device=device
    )
    return theme_classifier

In [90]:
theme_classifier = load_model(device)

/home/kelvin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [91]:
theme_list =['friendship','hope','sacrifice','battle','self development','betrayal','love','dialogue']

In [92]:
theme_classifier(
    'I gave him a right hook then a left job',
    theme_list,
    multi_label=True
)

{'sequence': 'I gave him a right hook then a left job',
 'labels': ['battle',
  'self development',
  'sacrifice',
  'betrayal',
  'hope',
  'dialogue',
  'friendship',
  'love'],
 'scores': [0.8370530605316162,
  0.6021761298179626,
  0.11033694446086884,
  0.045420169830322266,
  0.03848111256957054,
  0.03672131896018982,
  0.00513658719137311,
  0.0022709225304424763]}

# II. Loading dataset

In [93]:
files = glob('../data/Subtitles/*.ass')

In [94]:
def load_subtitles_data(dataset_path):
    subtitle_paths = glob(dataset_path+'/*.ass')

    scripts = []
    episode_number = []

    for path in subtitle_paths:

        with open(path, 'r') as file:
            lines = file.readlines()[27:]
            lines = [ ','.join(line.split(',')[9:]) for line in lines]
        lines = [line.replace('\\N', ' ') for line in lines]
        script = ' '.join(lines)

        episode = int(path.split('-')[-1].split('.')[0].strip())

        scripts.append(script)
        episode_number.append(episode)

    return pd.DataFrame.from_dict({'episode':episode_number,'script':scripts})
     

In [95]:
dataset_path = '../data/Subtitles/'
df = load_subtitles_data(dataset_path)

In [96]:
df.head()

,episode,script
0,119,Connecting old words that have been used up\n ...
1,126,Connecting old words that have been used up\n ...
2,209,"Fly into the wavy and twisted sky, into your h..."
3,183,"Rock away your existence,\n Shouting that you ..."
4,53,"Press down hard on the gas\n That’s right, the..."


# III. Run Model


In [97]:
script = str(df[df['episode'] == 1]['script'])

In [98]:
script = df[df['episode'] == 1]['script'].values[0]
script_sentences = sent_tokenize(script)

In [99]:
#Batch sentence
sentence_batch_size = 20
script_batches = []
for index in range(0, len(script_sentences), sentence_batch_size):
    sent = ' '.join(script_sentences[index:index + sentence_batch_size])
    script_batches.append(sent)

In [100]:
theme_output = theme_classifier(
    script_batches[:2],
    theme_list,
    multi_label=True
)

In [101]:
theme_output

[{'sequence': "A long time ago, a powerful demon fox appeared with nine tails. With its powerful tails,\n it could smash mountains and create tidal waves. A band of Ninjas rose to defend their village from attack. We have to wait until the Fourth Hokage gets here! We can't let it get any closer to our village! One great Ninja was able to imprison the monster,\n but died in the process. This Ninja was known as… the Fourth Hokage. Naruto! Why did you do such a thing?! You're really gonna get it this time! I don't care! You know your problem? You can't do the things I do! Only I can do this! I'm better than all of you! Believe it! There's a problem, sir! Lord Hokage! What is it? Did that Naruto do something again?",
  'labels': ['dialogue',
   'betrayal',
   'battle',
   'sacrifice',
   'self development',
   'hope',
   'friendship',
   'love'],
  'scores': [0.9800742268562317,
   0.9396904110908508,
   0.8546884059906006,
   0.7349812984466553,
   0.7284976243972778,
   0.199097707867622

In [102]:
# Wrangle output
themes = {}
for output in theme_output:
    for label, score in zip(output['labels'],output['scores']):
        if label not in themes:
            themes[label] = []
        themes[label].append(score)

In [103]:
themes

{'dialogue': [0.9800742268562317, 0.9370126724243164],
 'betrayal': [0.9396904110908508, 0.6457259654998779],
 'battle': [0.8546884059906006, 0.6581317186355591],
 'sacrifice': [0.7349812984466553, 0.6258842349052429],
 'self development': [0.7284976243972778, 0.8678209185600281],
 'hope': [0.19909770786762238, 0.20423933863639832],
 'friendship': [0.059223175048828125, 0.086033396422863],
 'love': [0.040261924266815186, 0.02802056074142456]}

In [104]:
def get_themes_inference(script):
    script_sentences = sent_tokenize(script)

    #Batch sentence
    sentence_batch_size = 20
    script_batches = []
    for index in range(0, len(script_sentences), sentence_batch_size):
        sent = ' '.join(script_sentences[index:index + sentence_batch_size])
        script_batches.append(sent)

    #Run Model
    theme_output = theme_classifier(
        script_batches[:2],
        theme_list,
        multi_label=True
    )

    # Wrangle output
    themes = {}
    for output in theme_output:
        for label, score in zip(output['labels'],output['scores']):
            if label not in themes:
                themes[label] = []
            themes[label].append(score)

    themes = {key: np.mean(np.array(value)) for key, value in themes.items()}
    return themes

In [106]:
df.head(2)

{'dialogue': 0.958543449640274,
 'betrayal': 0.7927081882953644,
 'battle': 0.7564100623130798,
 'sacrifice': 0.6804327666759491,
 'self development': 0.798159271478653,
 'hope': 0.20166852325201035,
 'friendship': 0.07262828573584557,
 'love': 0.03414124250411987}